In [1]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
sqlContext

In [2]:
! hdfs dfs -rm /user/cloudera/train.csv
! tail -n +2 ./small3.train.csv | hdfs dfs -put - /user/cloudera/train.csv

rm: `/user/cloudera/train.csv': No such file or directory


In [3]:
file = sc.textFile("hdfs:/user/cloudera/train.csv")
file.count()

9999

In [4]:
words = file.map(lambda x: x.split(','))
words

PythonRDD[3] at RDD at PythonRDD.scala:43

In [5]:
words.take(1)

[['1000009418151094273',
  '0',
  '14102100',
  '1005',
  '0',
  '1fbe01fe',
  'f3845767',
  '28905ebd',
  'ecad2386',
  '7801e8d9',
  '07d7df22',
  'a99f214a',
  'ddd2926e',
  '44956a24',
  '1',
  '2',
  '15706',
  '320',
  '50',
  '1722',
  '0',
  '35',
  '-1',
  '79']]

In [6]:
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)

words.count()

9999

In [7]:
df = sqlContext.createDataFrame(words)
df.show(2)

+--------------------+---+--------+----+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+---+---+-----+---+---+----+---+---+------+---+
|                  _1| _2|      _3|  _4| _5|      _6|      _7|      _8|      _9|     _10|     _11|     _12|     _13|     _14|_15|_16|  _17|_18|_19| _20|_21|_22|   _23|_24|
+--------------------+---+--------+----+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+---+---+-----+---+---+----+---+---+------+---+
| 1000009418151094273|  0|14102100|1005|  0|1fbe01fe|f3845767|28905ebd|ecad2386|7801e8d9|07d7df22|a99f214a|ddd2926e|44956a24|  1|  2|15706|320| 50|1722|  0| 35|    -1| 79|
|10000169349117863715|  0|14102100|1005|  0|1fbe01fe|f3845767|28905ebd|ecad2386|7801e8d9|07d7df22|a99f214a|96809ac8|711ee120|  1|  0|15704|320| 50|1722|  0| 35|100084| 79|
+--------------------+---+--------+----+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+---+---+-----+-

In [8]:
df.registerTempTable("train")
sqlContext.sql("select count(*) from train").show()

+----+
| _c0|
+----+
|9999|
+----+



In [9]:
%time sqlContext.sql("select _2, count(*) from train group by _2").show()

+---+----+
| _2| _c1|
+---+----+
|  0|8293|
|  1|1706|
+---+----+

CPU times: user 17 ms, sys: 15 ms, total: 32 ms
Wall time: 1.92 s


In [10]:
df.write.parquet("train_parquet")

AnalysisException: path hdfs://quickstart.cloudera:8020/user/cloudera/train_parquet already exists.;

In [11]:
df_parquet = sqlContext.read.load("train_parquet")
df_parquet

DataFrame[_1: string, _2: string, _3: string, _4: string, _5: string, _6: string, _7: string, _8: string, _9: string, _10: string, _11: string, _12: string, _13: string, _14: string, _15: string, _16: string, _17: string, _18: string, _19: string, _20: string, _21: string, _22: string, _23: string, _24: string]

In [12]:
%time df.groupBy("_2").count().show()

+---+-----+
| _2|count|
+---+-----+
|  0| 8293|
|  1| 1706|
+---+-----+

CPU times: user 10 ms, sys: 16 ms, total: 26 ms
Wall time: 792 ms
